In [1]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.adapters import AdapterArguments, AdapterTrainer, AdapterConfigBase, AutoAdapterModel, setup_adapter_training
from transformers import BertTokenizer, BertModelWithHeads, AdapterConfig, EvalPrediction, TextClassificationPipeline
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

from torch.utils.data import DataLoader, TensorDataset
import torch

from pdb import set_trace
import transformers.adapters.composition as ac
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}

adapter_info = {'cola': {'load_adapter': 'lingaccept/cola@ukp', 'adapter_config': 'pfeiffer'},
                'sst2': {'load_adapter': 'sentiment/sst-2@ukp', 'adapter_config': 'pfeiffer'},
                'mrpc': {'load_adapter': 'sts/mrpc@ukp',        'adapter_config': 'pfeiffer'},
                'qqp' : {'load_adapter': 'sts/qqp@ukp',         'adapter_config': 'pfeiffer'},
                'stsb': {'load_adapter': 'sts/sts-b@ukp',       'adapter_config': 'pfeiffer'},
                
                'rotten_tomatoes': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-rotten_tomatoes', 'adapter_config': 'pfeiffer'},
                'imdb': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-imdb', 'adapter_config': 'pfeiffer'},
                'yelp_polarity': {'load_adapter': 'AdapterHub/bert-base-uncased-pf-yelp_polarity', 'adapter_config': 'pfeiffer'},
               }

eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

metric_dict = {'rotten_tomatoes': 'sst2', 'imdb': 'sst2', 'yelp_polarity': 'sst2'}

In [2]:
task_name_1 = 'sst2'
task_name_2 = 'stsb'
model_name_or_path = 'bert-base-uncased'
pad_to_max_length = True
max_seq_length = 128
output_dir = f'./tmp/parallel_training'
load_adapter_1 = adapter_info[task_name_1]['load_adapter']
load_adapter_2 = adapter_info[task_name_2]['load_adapter']
adapter_config_1 = AdapterConfigBase.load(adapter_info[task_name_1]['adapter_config'])
adapter_config_2 = AdapterConfigBase.load(adapter_info[task_name_2]['adapter_config'])

BATCH_SIZE = 2048 

set_seed(0)

In [3]:
raw_datasets_1 = load_dataset("glue", task_name_1) if task_name_1 in is_glue else load_dataset(task_name_1)
raw_datasets_2 = load_dataset("glue", task_name_2) if task_name_2 in is_glue else load_dataset(task_name_2)

In [4]:
def get_num_labels(task_name, raw_datasets):
    # Labels
    if task_name_1 is not None:
        is_regression = task_name == "stsb"
        if not is_regression:
            label_list = raw_datasets["train"].features["label"].names
            num_labels = len(label_list)
        else:
            num_labels = 1
    else:
        # Trying to have good defaults here, don't hesitate to tweak to your needs.
        is_regression = raw_datasets["train"].features["label"].dtype in ["float32", "float64"]
        if is_regression:
            num_labels = 1
        else:
            # A useful fast method:
            # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
            label_list = raw_datasets["train"].unique("label")
            label_list.sort()  # Let's sort it for determinism
            num_labels = len(label_list)
    return num_labels, is_regression

num_labels_1, is_regression_1 = get_num_labels(task_name_1, raw_datasets_1)
num_labels_2, is_regression_2 = get_num_labels(task_name_2, raw_datasets_2)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
)

def get_data(task_name, raw_datasets):
    sentence1_key, sentence2_key = task_to_keys[task_name]

    if pad_to_max_length:
        padding = "max_length"

    def preprocess_function(examples):    
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    
        # Map labels to IDs (not necessary for GLUE tasks)
        # if label_to_id is not None and "label" in examples:
            # result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        result["label"] = [(l if l != -1 else -1) for l in examples["label"]]
        return result
    
    raw_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        desc="Running tokenizer on dataset",
    )

    return raw_datasets

In [6]:
dataset1 = get_data(task_name_1, raw_datasets_1)
dataset2 = get_data(task_name_2, raw_datasets_2)

train_dataset_1 = dataset1['train']
train_dataset_2 = dataset2['train']

eval_dataset_1 = dataset1[eval_data_dict[task_name_1]] if task_name_1 in eval_data_dict else dataset1['validation']  
eval_dataset_2 = dataset2[eval_data_dict[task_name_2]] if task_name_2 in eval_data_dict else dataset2['validation']  

In [7]:
def get_compute_metrics(task_name, is_regression):
    if task_name in metric_dict:
        metric = evaluate.load("glue", metric_dict[task_name])
    else:
        metric = evaluate.load("glue", task_name)
    
    def compute_metrics(p: EvalPrediction):
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
        if task_name is not None:
            result = metric.compute(predictions=preds, references=p.label_ids)
            if len(result) > 1:
                result["combined_score"] = np.mean(list(result.values())).item()
            return result
        elif is_regression:
            return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
        else:
            return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

    return compute_metrics

compute_metrics_1 = get_compute_metrics(task_name_1, is_regression_1)
compute_metrics_2 = get_compute_metrics(task_name_2, is_regression_2)

In [8]:
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# We use the AutoAdapterModel class here for better adapter support.
model = AutoAdapterModel.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=False
)


adapter1 = model.load_adapter(load_adapter_1)
adapter2 = model.load_adapter(load_adapter_2)

model.active_adapters = ac.Parallel(adapter1, adapter2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No valid map of labels in label2id. Falling back to default (num_labels=2). This may cause errors during loading!


In [10]:
print(model.adapter_summary())

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
sst-2                    bottleneck          894,528       0.817       1       1
sts-b                    bottleneck          894,528       0.817       1       1
--------------------------------------------------------------------------------
Full model                               109,482,240     100.000               1


In [11]:
model.active_head

['sst-2', 'sts-b']

In [12]:
print(task_name_1)

# Convert data to tensors and move to device
input_ids = torch.tensor(eval_dataset_1['input_ids'])
token_type_ids = torch.tensor(eval_dataset_1['token_type_ids'])
attention_mask = torch.tensor(eval_dataset_1['attention_mask'])
labels = torch.tensor(eval_dataset_1['label'])

# Create a DataLoader
dataset = TensorDataset(input_ids, token_type_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

model = model.to(device)
model.eval()

all_logits = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(dataloader):
        batch = [b_input.to(device) for b_input in batch]
        b_input_ids, b_token_type_ids, b_attention_mask, b_labels = batch
        inputs = {
            'input_ids': b_input_ids,
            'token_type_ids': b_token_type_ids,
            'attention_mask': b_attention_mask
        }
        
        output1, output2 = model(**inputs)
        logits = output1.logits
        all_logits.append(logits.cpu().numpy())
        all_labels.append(b_labels.cpu().numpy())

# Get predictions
prediction1 = np.concatenate(all_logits, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Evaluate
compute_results = compute_metrics_1(EvalPrediction(predictions=prediction1, label_ids=all_labels))
print(compute_results)


sst2


100%|██████████| 1/1 [00:01<00:00,  1.47s/it]

{'accuracy': 0.908256880733945}


In [14]:
print(task_name_2)

# Convert data to tensors and move to device
input_ids = torch.tensor(eval_dataset_2['input_ids'])
token_type_ids = torch.tensor(eval_dataset_2['token_type_ids'])
attention_mask = torch.tensor(eval_dataset_2['attention_mask'])
labels = torch.tensor(eval_dataset_2['label'])

# Create a DataLoader
dataset = TensorDataset(input_ids, token_type_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

all_logits = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(dataloader):
        batch = [b_input.to(device) for b_input in batch]
        b_input_ids, b_token_type_ids, b_attention_mask, b_labels = batch
        inputs = {
            'input_ids': b_input_ids,
            'token_type_ids': b_token_type_ids,
            'attention_mask': b_attention_mask
        }
        
        output1, output2 = model(**inputs)
        logits = output2.logits
        all_logits.append(logits.cpu().numpy())
        all_labels.append(b_labels.cpu().numpy())

# Get predictions
prediction1 = np.concatenate(all_logits, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Evaluate
compute_results = compute_metrics_2(EvalPrediction(predictions=prediction1, label_ids=all_labels))
print(compute_results)


stsb


100%|██████████| 1/1 [00:02<00:00,  2.41s/it]

{'pearson': 0.8260246487510453, 'spearmanr': 0.8410607771863062, 'combined_score': 0.8335427129686758}
